### Redes Neuronales para Predicción de Precios de Acciones

Predecir la dirección del precio de una acción (subida/bajada) basado en:
- Nivel de cambio del precio de la posición
- Nivel de cambio del volumen transado

La red neuronal aprenderá patrones en estas variables para clasificar movimientos futuros del mercado.

In [1]:
import yfinance as yf
import torch
from torch import nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = yf.download('BAP', period='1000d', interval='1d')
data['Price_Change'] = data['Close'].pct_change()
data['Volume_Change'] = data['Volume'].pct_change()

#data['Price_Change']


/tmp/ipykernel_10587/3037657050.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('BAP', period='1000d', interval='1d')
[*********************100%***********************]  1 of 1 completed


In [2]:
data['Target'] = (data['Price_Change'].shift(-1) > 0).astype(int)
data = data.iloc[1:-1] # remuevo la primera y la última fila

In [18]:
# Stock Dataset
from torch.utils.data import Dataset

class StockDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype = torch.long)
        
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        return self.features[index], self.targets[index]

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

X = data[['Price_Change', 'Volume_Change']].values # type: ignore
Y = data['Target'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=.2, random_state=42)

train_dataset = StockDataset(X_train, Y_train)
test_dataset = StockDataset(X_test, Y_test)

In [20]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=True)

In [25]:
class Net(nn.Module):
    def __init__(self, input_size, hidde_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidde_size = hidde_size
        self.fc1 = nn.Linear(input_size, hidde_size)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(hidde_size, 2)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return x

In [26]:
model = Net(2, 15)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [ ]:
epochs = 3

for epoch in range(epochs):
    model.train()
    for batch_x, batch_y in train_dataloader:
        y_hat = model(batch_x)
        print(f'yhat {y_hat.shape} vs {batch_y.shape}')
        print(f'target is {batch_y}')
        loss = criterion(y_hat, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        

In [ ]:

model.eval()
correct = 0
total = 0

with torch.no_grad():    
    for batch_x, batch_y in test_dataloader:    
        y_pred_test = model(batch_x)
        print(f'y_pred_test is {y_pred_test}')
        predicted_class = torch.argmax(y_pred_test, dim=1)
        total += batch_y.size(0)
        correct = (predicted_class == batch_y).sum().item()
    
    accuracy = correct / total
    print(f'Test accuracy : {accuracy:.4f}')
    

Reset model weight

In [ ]:
for name, param in model.named_parameters():
    print(f"{name} : {param.data.mean():.3f}")

fc1.weight : 0.088
fc1.bias : 0.066
fc2.weight : 0.120
fc2.bias : 0.360


In [42]:
def reset_weights(model):
    for layer in model.children():
        if hasattr(layer, 'reset_paramters'):
            layer.reset_parameters()

reset_weights(model)

print("model parameter after reset:")

for name, param in model.named_parameters():
    print(f"{name} : {param.data.mean():.4f}")




model parameter after reset:
fc1.weight : 0.1547
fc1.bias : -0.9573
fc2.weight : -0.2598
fc2.bias : 0.8145
